# Informe Técnico: Desarrollo de un Modelo Predictivo de Vida Útil Restante (RUL) para Bogies de Tren

---

## Índice <a id="indice"></a>


* [1.0 Introducción al Mantenimiento Predictivo](#10-introduccion)
* [2.0 Preparación y Análisis Exploratorio de Datos (EDA)](#20-eda)
* [3.0 Ingeniería de Características y Definición de RUL](#30-feature-engineering)
* [4.0 Diseño del Pipeline y Estrategia de Validación](#40-pipeline)
* [5.0 Entrenamiento y Evaluación Comparativa](#50-entrenamiento)
* [6.0 Análisis de Resultados y Selección del Modelo Final](#60-resultados)
* [7.0 Sistema Complementario de Detección por Imagen](#70-vision)
* [8.0 Conclusiones](#80-conclusiones)
* [9.0 Limitaciones y Próximos Pasos](#90-limitaciones)

---

## 1.0 Introducción al Mantenimiento Predictivo <a id="10-introduccion"></a>

El mantenimiento predictivo representa un cambio de paradigma en la industria ferroviaria, pasando de un enfoque reactivo a una estrategia proactiva guiada por datos. Su implementación impacta directamente en la **seguridad**, aumenta la **disponibilidad de la flota** y reduce costos por paradas no planificadas.

Este informe detalla el desarrollo de un sistema para predecir la **Vida Útil Restante (RUL)** de los bogies, permitiendo anticipar la degradación con una precisión sin precedentes.

[Volver al índice](#indice)

---

## 2.0 Preparación y Análisis Exploratorio de Datos (EDA) <a id="20-eda"></a>

La calidad de los datos es el pilar del modelo. El dataset original contaba con 200,000 filas y un severo desbalance (solo 1.5% de fallos).

### Proceso de Limpieza y Tratamiento

* **Temporales:** Conversión a `datetime` e imputación de nulos mediante *ffill/bfill* por `bogie_id`.

* **Variables Categóricas:** Imputación de `curve_class` con la moda ("straight").

* **Sensores Físicos:**

  * *Clipping operativo:* Velocidad (0–160 km/h) y Carga (0–25 t).
  * *Temperaturas:* Imputación por mediana de cada bogie y acotación a rangos realistas (ej. 25–60 °C para el bogie).

* **Dataset Final:** 197,000 registros limpios tras eliminar nulos críticos en `target_fault`.

### Hallazgos Visuales

* **Distribuciones:** Las vibraciones RMS muestran una fuerte asimetría positiva, indicando eventos anómalos esporádicos.
* **Correlación con Fallo:** Los diagramas de caja revelaron que el nivel de vibración por sí solo **no es suficiente** para predecir un fallo, justificando la necesidad de ingeniería de características avanzada.

[Volver al índice](#indice)

---

## 3.0 Ingeniería de Características y Definición de RUL <a id="30-feature-engineering"></a>

Se transformaron los datos brutos en señales físicas de desgaste:

* **Ratios contextuales:** `vib_load_ratio`, `vib_speed_ratio` para normalizar vibración según la exigencia operativa.
* **Diferenciales térmicos:** `bogie_wheel_temp_diff` para detectar asimetrías mecánicas.
* **Estrés acumulado:** `over_temp_duration` para cuantificar el tiempo por encima de umbrales críticos.

### Definición de la Variable Objetivo: `RUL_steps`

Se pivotó de un problema de clasificación binaria a uno de regresión. **RUL_steps** representa el número de registros restantes hasta el fallo (rango 0–272).

| Nivel de Riesgo | Rango (RUL_steps) | Interpretación Operativa           |
| --------------- | ----------------- | ---------------------------------- |
| **MUY CRÍTICO** | 0–1               | Fallo inminente. Parada inmediata. |
| **CRÍTICO**     | 2–10              | Intervención en < 24 h.            |
| **ALTO RIESGO** | 11–30             | Programar inspección (1–3 días).   |
| **BAJO RIESGO** | > 30              | Monitoreo rutinario.               |

[Volver al índice](#indice)

---

## 4.0 Diseño del Pipeline y Estrategia de Validación <a id="40-pipeline"></a>

Para asegurar la capacidad de generalización del modelo se aplicaron las siguientes estrategias:

* **División por `train_id`:** Split 80 % entrenamiento / 20 % validación a nivel de tren, evitando fuga de información temporal.
* **Balanceo:** Submuestreo (*undersampling*) de la clase **BAJO RIESGO** para mejorar la sensibilidad en eventos críticos.
* **Ponderación de muestras:** Uso de `sample_weight`, asignando peso **8.0** a casos muy críticos frente a **1.0** en casos normales.

[Volver al índice](#indice)

---

## 5.0 Entrenamiento y Evaluación Comparativa <a id="50-entrenamiento"></a>

La métrica principal fue el **MAE**, evaluado específicamente en el rango crítico (RUL < 50):

1. **Modelos lineales:** MAE ≈ 19.32. Incapaces de capturar relaciones no lineales.
2. **Ensamblados (XGBoost / GBM):** Mejora significativa (MAE ≈ 12.88).
3. **Random Forest optimizado:** MAE ≈ 12.15.
4. **Random Forest + PCA:** Reducción a 13 componentes (96.8 % de varianza explicada), obteniendo el mejor rendimiento.

[Volver al índice](#indice)

---

## 6.0 Análisis de Resultados y Selección del Modelo Final <a id="60-resultados"></a>

El modelo final seleccionado es **Random Forest con PCA**, alcanzando un MAE de **12.10**.

### Interpretación Operativa del Error

* **Muestreo cada 10 min:** ±2 h de precisión.
* **Muestreo cada 60 min:** ±12 h de precisión.

### Variables Más Influyentes

1. `bogie_health_score` (indicador holístico).
2. `vib_load_ratio` (vibración normalizada por carga).
3. `external_temp_c` (influencia ambiental).

[Volver al índice](#indice)

---

## 7.0 Sistema Complementario de Detección por Imagen <a id="70-vision"></a>

Se incorporó un sistema de visión artificial para detectar defectos superficiales:

* **Arquitectura:** YOLOv8 *fine-tuned*.
* **Entrenamiento:** 50 épocas con imágenes de 640×640 px.
* **Exportación:** ONNX y PKL para integración en entornos industriales.

[Volver al índice](#indice)

---

## 8.0 Conclusiones <a id="80-conclusiones"></a>

El proyecto entrega un pipeline integral que transforma datos de sensores en decisiones de mantenimiento. La combinación de un modelo RUL preciso con visión artificial permite cubrir tanto degradación interna como defectos visuales, optimizando seguridad y eficiencia operativa.

[Volver al índice](#indice)

---

## 9.0 Limitaciones y Próximos Pasos <a id="90-limitaciones"></a>

### Limitaciones Actuales

* Dependencia de la frecuencia de muestreo.
* Incremento del error en horizontes de predicción largos.
* No integración actual de históricos de mantenimiento.

### Próximos Pasos

* Redefinir el target en unidades físicas (km, días).
* Explorar modelos temporales (LSTM, Transformers).
* Integrar meteorología avanzada y topografía de vía.

[Volver al índice](#indice)
